## This notebook will focus on the collection of training data for the project based on our four areas of focus.

Areas of focus:
 - Battery materials.
 - Photovoltaic materials.
 - Carbon capture.
 - Photocatalytic materials

## A: Battery materials.